<a href="https://colab.research.google.com/github/tkoide01/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar 

--2021-06-22 19:23:16--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2021-06-22 19:23:16 (11.5 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Apps_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1774101|R3PMQXEVYR4ZT1|B00DVKOYBM|     263046246|         8 Ball Pool|     Mobile_Apps|          3|            0|          0|   N|                Y|                 Fun|It is a fun game ...| 2015-08-31|
|         US|   13679234|R1I1K3EO98EMUV|B00R1IA4RS|     128868854|Christmas Cookie ...|     Mobile_Apps|          5|    

In [5]:
# Filter the dataset for total_votes being greater than or equal to 20
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vote_count = vine_df.filter("total_votes >= 20")

In [6]:
# Create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
filtered_table = vote_count.filter(vote_count["helpful_votes"]/vote_count["total_votes"]>=0.5)
filtered_table.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1Z9QZRQJ1F8JX|          1|           17|         22|   N|                Y|
|R31PAYBVBP8W62|          1|           54|         81|   N|                Y|
|R33GCMAT1K5DX8|          2|          141|        190|   N|                Y|
|R3MIL83RY4SMS4|          1|           27|         38|   N|                Y|
|R37T9FXGG07XOM|          1|           24|         28|   N|                Y|
|R2E9ZHX72173PZ|          1|           14|         21|   N|                Y|
|R2I6YA0BVG78P6|          3|           59|         70|   N|                Y|
|R1NB53BPRH24RK|          5|           15|         26|   N|                Y|
|R3AX1CCCZH20C5|          5|           17|         26|   N|                Y|
|R1FOM4K4S3S1A3|          4|           19|         22|   N|     

In [22]:
filtered_table.count()

129516

In [14]:
# Create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
vine_table = filtered_table.filter(filtered_table["vine"] == "Y")
vine_table.count()

0

In [15]:
# Create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'N'
non_vine_table = filtered_table.filter(filtered_table["vine"] == "N")
non_vine_table.count()

129516

In [17]:
# Filter the above tables and count the Vine reviews that are 5 star.
five_star_vine= vine_table.filter(filtered_table["star_rating"]== 5)
five_star_vine.count()

0

In [16]:
# Filter the above tables and count the non-Vine reviews that are 5 star.
five_star_non_vine= non_vine_table.filter(filtered_table["star_rating"]== 5)
five_star_non_vine.count()

59278

In [18]:
# Determine the percentage of Vine reviews that are 5 stars
five_star_vine.count() / vine_table.count() 

ZeroDivisionError: ignored

In [21]:
# Determine the percentage of non-Vine reviews that are 5 stars
five_star_non_vine.count() / non_vine_table.count()

0.4576886253435869

In [13]:
# Calculate the percentage of five star reviews
five_star.count() / filtered_table.count()

0.4576886253435869

In [ ]:
# Calculate the percentage of paid reviews
five_star.filter(five_star["verified_purchase"] == "Y").count()/filtered_table.filter(filtered_table["verified_purchase"]== "Y").count()

In [ ]:
# Calculate percentage of unpaid reviews
five_star.filter(five_star["verified_purchase"] == "N").count()/filtered_table.filter(filtered_table["verified_purchase"]== "N").count()